## Análise descritiva da PNAD COVID

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
covid = pd.read_parquet('../Dados/pnad_covid/pnad_covid_filtrada.parquet')

### Variáveis da PNAD Covid-19 utilizadas no estudo:
- Gênero (A003)
- Cor-raça (A004)
- Faixa etária (A002)
- Escolaridade (A005)
- Região (UF)
- Formalidade (C007)
- Atividade (C007)
- Ocupação (C001, C002, C003)
- Trabalho remoto (C001 e C013)


In [13]:
covid.groupby(['Mes','Ocupado'])['Ocupado'].count()

Mes  Ocupado
5    Sim        104559
     Não        167799
6    Sim        118161
     Não        179018
7    Sim        123239
     Não        176294
8    Sim        129988
     Não        171505
9    Sim        133498
     Não        169272
10   Sim        134187
     Não        163821
11   Sim        135860
     Não        163480
Name: Ocupado, dtype: int64

In [4]:
covid.rename(columns={
    'V1013':'Mes',
    'A003':'Sexo',
    'A004':'Cor',
    'A005':'Escolaridade',
    'C001':'Ocupado',
    'C013':'Trabalho remoto'
}, inplace=True
)

covid['Mes'] = covid['Mes'].astype(int)

#### Distribuição do trabalho remoto por características individuais

Ocupação

In [5]:
print('Total de indivíduos ocupados no mês de referência (Milhões de pessoas)')

pd.DataFrame(
    pd.crosstab(
        covid['Ocupado'], 
        covid['Mes'], 
        values=covid['V1032'], 
        aggfunc='sum',
        margins=True,
        margins_name='Total (PEA)'
    )/1000000 #Divisão por 1 milhão para facilitar a visualização
).drop('Total (PEA)', axis=1)

Total de indivíduos ocupados no mês de referência (Milhões de pessoas)


Mes,5,6,7,8,9,10,11
Ocupado,,,,,,,
Sim,63.780851,67.070105,70.137901,73.840721,75.952550,77.819648,78.675024
Não,97.561933,94.717452,91.836620,88.385302,86.717227,84.972345,84.421591
Total (PEA),161.342784,161.787557,161.974521,162.226022,162.669777,162.791993,163.096615


Trabalho remoto por genêro ao longo dos meses

In [9]:
print('Total de indivíduos que realizaram ou não trabalho remoto segmentado por genêro (Milhões de pessoas)')

pd.DataFrame(
    pd.crosstab(
        [covid['Sexo'], covid['Trabalho remoto']], 
        covid['Mes'], 
        values=covid['V1032'], 
        aggfunc='sum',
        margins=True,
        margins_name='Total'
    )/1000000
).drop('Total', axis=1)

Total de indivíduos que realizaram ou não trabalho remoto segmentado por genêro (Milhões de pessoas)


Mes                             5          6          7          8          9  \
Sexo   Trabalho remoto                                                          
Homem  Sim               3.902975   3.755761   3.619534   3.512866   3.370069   
       Não               2.118613   2.017542   1.982421   2.005666   2.058747   
Mulher Sim               4.500181   4.657520   4.530713   4.606107   4.454260   
       Não               0.719061   0.636421   0.560021   0.538248   0.519028   
Total                   11.240829  11.067245  10.692689  10.662888  10.402103   

Mes                           10        11  
Sexo   Trabalho remoto                      
Homem  Sim              3.179638  3.003605  
       Não              2.065818  2.046403  
Mulher Sim              4.182982  4.102772  
       Não              0.527442  0.524657  
Total                   9.955881  9.677438